In [1]:
import re
import os
from random import shuffle
from collections import defaultdict

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.callbacks import LambdaCallback, ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional, Embedding

Using TensorFlow backend.


In [10]:
DIR_DATASET = './dataset-musica/sertanejo/artistas'
regex_pontos = re.compile(r'(,|\:|\.|\!|\?|\\n)')
regex_feat = re.compile(r'\[[\w\s]+\]|\-|\([\w\s]+\)')

In [11]:
dataset = []
e = []
for pasta in os.listdir(DIR_DATASET):
#     if pasta == 'funk':
#         continue

    caminho = f'{DIR_DATASET}/{pasta}'
    for arquivo in os.listdir(caminho):
        musica = open(f"{caminho}/{arquivo}").read().lower().replace('"', '')
        musica = regex_feat.sub('', regex_pontos.sub(r' \1 ', musica))
        
#         repeticao = defaultdict(int)
#         for palavra in musica.split():
#             repeticao[palavra] += 1
        dataset.append(musica)
#         if len(sorted(repeticao.items(), key=lambda x: x[1])[-1][0]) > 3:
#             
#         else:
#             print(sorted(repeticao.items(), key=lambda x: x[1])[-1][0])

In [ ]:
shuffle(dataset)

In [12]:
palavras = [palavra for palavra in ''.join(dataset).split()]
palavras_unq = set(palavras)
print('Quantidade de palavras: ', len(palavras))
print('Palavras unicas:', len(palavras_unq))

Quantidade de palavras:  1015186
Palavras unicas: 28599


In [13]:
palavras[1000:1005]

['tamanho', 'p', 'que', 'é', 'pra']

In [14]:
frequencia_palavras = defaultdict(int)
for palavra in palavras:
    frequencia_palavras[palavra] += 1 

In [15]:
freq = sorted(frequencia_palavras.items(), key=lambda x: x[1], reverse=True)
print('TOP 10 Palavras mais usadas:', freq[:10], sep='\n', end='\n\n')
print('TOP 20 Palavras menos usadas:', freq[-21:])

TOP 10 Palavras mais usadas:
[(',', 37924), ('que', 32086), ('eu', 26450), ('o', 24753), ('e', 24034), ('de', 23914), ('a', 23114), ('não', 20040), ('é', 15081), ('me', 13231)]

TOP 20 Palavras menos usadas: [('zebuzada', 1), ('estalando', 1), ('riscou', 1), ('seguirá', 1), ('selecionava', 1), ('tratamos', 1), ('anunciaram', 1), ('severa', 1), ('residir', 1), ('socorra', 1), ('coroinhas', 1), ('d`água', 1), ('ponteava', 1), ('serestas', 1), ('fraquejando', 1), ('aquecerte', 1), ('atirarse', 1), ('zagais', 1), ('impricá', 1), ('sinho', 1), ('cumbiná', 1)]


In [16]:
freq

[(',', 37924),
 ('que', 32086),
 ('eu', 26450),
 ('o', 24753),
 ('e', 24034),
 ('de', 23914),
 ('a', 23114),
 ('não', 20040),
 ('é', 15081),
 ('me', 13231),
 ('você', 12508),
 ('pra', 12008),
 ('um', 10539),
 ('se', 10362),
 ('.', 10148),
 ('meu', 10107),
 ('te', 9531),
 ('do', 8326),
 ('amor', 8028),
 ('no', 7892),
 ('na', 7643),
 ('com', 6538),
 ('da', 6223),
 ('em', 6150),
 ('seu', 6000),
 ('mais', 5783),
 ('só', 5644),
 ('minha', 5441),
 ('por', 5419),
 ('vai', 5347),
 ('vou', 5195),
 ('mas', 4904),
 ('uma', 4862),
 ('ela', 4241),
 ('tem', 4072),
 ('já', 3876),
 ('sem', 3824),
 ('coração', 3782),
 ('quando', 3673),
 ('foi', 3590),
 ('vida', 3437),
 ('quero', 3295),
 ('mim', 3189),
 ('!', 3061),
 ('tudo', 3014),
 ('quem', 2989),
 ('tá', 2970),
 ('sou', 2955),
 ('gente', 2737),
 ('sua', 2654),
 ('os', 2646),
 ('sei', 2628),
 ('?', 2590),
 ('ai', 2573),
 ('ser', 2568),
 ('bem', 2518),
 ('tô', 2511),
 ('dia', 2500),
 ('as', 2380),
 ('como', 2254),
 ('nem', 2166),
 ('saudade', 2157),
 (

In [17]:
MIN_FREQ = 10

In [18]:
palavras_removidas = set()
for palavra, freq in frequencia_palavras.items():
    if freq < MIN_FREQ:
        palavras_removidas.add(palavra)

In [19]:
print(f'Palavras removidas (< {MIN_FREQ}): {len(palavras_removidas)}')

Palavras removidas (< 10): 22988


In [20]:
palavras_restantes = palavras_unq - palavras_removidas
print(f'Palavras restantes (>= {MIN_FREQ}): {len(palavras_restantes)}')

Palavras restantes (>= 10): 5611


In [21]:
palavra_index = dict((c, i) for i, c in enumerate(palavras_restantes))
index_palavra = dict((i, c) for i, c in enumerate(palavras_restantes))

In [22]:
print('chave (palavra)'.ljust(20), 'valor (index)')
for palavra, index in list(palavra_index.items())[:5]:
    print(palavra.ljust(20), index)

chave (palavra)      valor (index)
bebe                 0
honrado              1
bueno                2
moderno              3
vezes                4


In [23]:
print('chave (index)'.ljust(20), 'valor (palavra)')
for index, palavra in list(index_palavra.items())[:5]:
    print(str(index).ljust(20), palavra)

chave (index)        valor (palavra)
0                    bebe
1                    honrado
2                    bueno
3                    moderno
4                    vezes


### Montar lista com as palavras e próxima palavra

In [24]:
STEP = 1
TAM_SEQ = 5
frases = []
proxima_palavra = []
frases_ignoradas = 0

for index in range(0, len(palavras)-TAM_SEQ, STEP):
    frase = palavras[index:index + TAM_SEQ + 1]
    if len(set(frase) & palavras_removidas) == 0:
        frases.append(frase[:-1])
        proxima_palavra.append(frase[-1])
    else:
        frases_ignoradas += 1

In [25]:
print(f'Quantidade de frases: {len(frases)}')
print(f'Quantidade de frases ignoradas: {frases_ignoradas}')

Quantidade de frases: 752685
Quantidade de frases ignoradas: 262496


In [26]:
def shuffle_and_split_training_set(sentences_original, next_original, percentage_test=2):
    # shuffle at unison
    print('Shuffling sentences')

    tmp_sentences = []
    tmp_next_word = []
    for i in np.random.permutation(len(sentences_original)):
        tmp_sentences.append(sentences_original[i])
        tmp_next_word.append(next_original[i])

    cut_index = int(len(sentences_original) * (1.-(percentage_test/100.)))
    x_train, x_test = tmp_sentences[:cut_index], tmp_sentences[cut_index:]
    y_train, y_test = tmp_next_word[:cut_index], tmp_next_word[cut_index:]

    print("Size of training set = %d" % len(x_train))
    print("Size of test set = %d" % len(y_test))
    return x_train, y_train, x_test, y_test

In [27]:
x_treino, y_treino, x_teste, y_teste = shuffle_and_split_training_set(frases, proxima_palavra)

Shuffling sentences
Size of training set = 737631
Size of test set = 15054


In [28]:
def generator(frases, proximas_palavras, palavra_index, batch_size, tam_seq):
    """ Gerar os batch de 32 para não sobrecarregar a memória ram.
    
    O resultado será um np.array com os index das palavras.    
    """
    index = 0
    while True:
        x = np.zeros((batch_size, tam_seq), dtype=np.int32) # np.array que vai armazenar as frases
        y = np.zeros((batch_size), dtype=np.int32) # np.array que vai armazenar a próxima palavra da frase

        for i in range(batch_size):
            for j, palavra in enumerate(frases[index % len(frases)]):
                x[i, j] = palavra_index[palavra]
            y[i] = palavra_index[proximas_palavras[index % len(frases)]]
            index = index + 1

        yield x, y

In [29]:
# Data generator for fit and evaluate
def get_model(palavras, ativacao, dropout=0.2, unidades=128, saida=1024):
    print('Build model...')
    model = Sequential()
    model.add(Embedding(input_dim=len(palavras), output_dim=saida))
    model.add(Bidirectional(LSTM(unidades, return_sequences=True)))
    model.add(Bidirectional(LSTM(unidades)))
    if dropout > 0:
        model.add(Dropout(dropout))
    model.add(Dense(len(palavras)))
    model.add(Activation(ativacao)) #softmax #sigmoid

    return model

In [30]:
#Functions from keras-team/keras/blob/master/examples/lstm_text_generation.py
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [31]:
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    arquivo_saida.write('\n----- Gerando texto após época: %d\n' % epoch)

    index_frase = np.random.randint(len(x_treino+x_teste))
    frase = (x_treino+x_teste)[index_frase]

    for diversity in [0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
        frase_cp = frase

        arquivo_saida.write('----- Diversity:' + str(diversity) + '\n')
        arquivo_saida.write('----- Generating with seed:\n"' + ' '.join(frase_cp) + '"\n')
        arquivo_saida.write(' '.join(frase_cp))

        for i in range(50): # quantidade de palavras
            x_pred = np.zeros((1, TAM_SEQ))
            for j, palavra in enumerate(frase_cp):
                x_pred[0, j] = palavra_index[palavra]

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_word = index_palavra[next_index]

            frase_cp = frase_cp[1:]
            frase_cp.append(next_word)

            arquivo_saida.write(" "+next_word)
        arquivo_saida.write('\n')
    arquivo_saida.write('='*80 + '\n')
    arquivo_saida.flush()

In [ ]:
BATCH_SIZE = 256

from datetime import datetime

ini = datetime.now()
model = get_model(palavras_restantes, 'sigmoid', dropout=0.3, unidades=128, saida=1024)
model.compile(loss='sparse_categorical_crossentropy', optimizer="adamax", metrics=['accuracy'])

file_path="./checkpoints/teste4-p%d_seq%d_freq%d-{epoch:02d}-{val_accuracy:.2f}.hdf5" \
            %(len(palavras_restantes), TAM_SEQ, MIN_FREQ)

print(file_path)

checkpoint = ModelCheckpoint(file_path, monitor='val_accuracy', save_best_only=True)
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
early_stopping = EarlyStopping(monitor='val_accuracy', patience=20)
callbacks_list = [checkpoint, print_callback, early_stopping]

arquivo_saida = open('log_treinamento.log', "w")
his = model.fit_generator(generator(x_treino, y_treino, palavra_index, BATCH_SIZE, TAM_SEQ),
                    steps_per_epoch=int(len(frases)/BATCH_SIZE) + 1,
                    epochs=700,
                    callbacks=callbacks_list,
                    validation_data=generator(x_teste, y_teste, palavra_index, BATCH_SIZE, TAM_SEQ),
                    validation_steps=int(len(x_teste)/BATCH_SIZE) + 1)
fim = datetime.now() - ini

Build model...
./checkpoints/teste4-p5611_seq5_freq10-{epoch:02d}-{val_accuracy:.2f}.hdf5


/home/monopoly/Envs/pi3/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/700
2941/2941 [==============================] - 68s 23ms/step - loss: 5.9646 - accuracy: 0.0522 - val_loss: 5.4856 - val_accuracy: 0.0876


/home/monopoly/Envs/pi3/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """


Epoch 2/700
2941/2941 [==============================] - 68s 23ms/step - loss: 5.3735 - accuracy: 0.1007 - val_loss: 5.1999 - val_accuracy: 0.1178
Epoch 3/700
2786/2941 [===========================>..] - ETA: 3s - loss: 5.0971 - accuracy: 0.1229

In [ ]:
print(fim)

In [ ]:
frase = "minha paixão".split()
for diversity in [0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    frase_cp = frase

    print('----- Diversity:' + str(diversity) + '\n')
    print('----- Generating with seed:\n"' + ' '.join(frase_cp) + '"\n')
    print(' '.join(frase_cp), end=' ')

    for i in range(50): # quantidade de palavras
        x_pred = np.zeros((1, 2))
        for j, palavra in enumerate(frase_cp):
            x_pred[0, j] = palavra_index[palavra]

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_word = index_palavra[next_index]

        frase_cp = frase_cp[1:]
        frase_cp.append(next_word)
        print(next_word, end=' ')

In [ ]:
print(his.history.keys())
# summarize history for accuracy
plt.plot(his.history['accuracy'])
plt.plot(his.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(his.history['loss'])
plt.plot(his.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()